In [1]:
import xml.etree.ElementTree as ET
import gzip
import pandas as pd
import csv
import duckdb
import numpy as np

## 100 pct

In [ ]:
pathToPlans100pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans100pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/departures_100pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "person", "link", "legMode"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "departure" and elem.attrib['legMode'] == "car"):
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)
            
        root.clear()

In [6]:
pathToPlans100pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans100pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/arrivals_100pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "person", "link", "legMode"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "arrival" and elem.attrib['legMode'] == "car"):
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)
            
        root.clear()

In [2]:
def networkInfoToDf(pathToNetwork):
    input = gzip.open(pathToNetwork, 'r')
    tree = ET.parse(input)
    root = tree.getroot()
    # convert network to data frame 
    ids = []
    length = []
    freespeed = []
    capacity = []
    type_of_link = []

    for links in root.findall('links'):
        for link in links:
            #print(link.tag, link.attrib)
            ids.append(link.attrib['id'])
            length.append(float(link.attrib['length']))
            freespeed.append(float(link.attrib['freespeed']))
            capacity.append(float(link.attrib['capacity']))
            type_counter = 0
            for child in link:
                for attr in child:
                    if (attr.attrib['name'] == "type"):
                        try:
                            type_of_link.append(attr.text)
                            type_counter = 1
                        except:
                            type_of_link.append('NA')
            if(type_counter == 0):
                type_of_link.append('NA')
    network_df = pd.DataFrame({'link_id': ids, 'length': length, 'freespeed': freespeed, 'capacity': capacity, 'type': type_of_link })
    return network_df

In [3]:
pathToNetwork = "/home/lola/Downloads/lausitz-v2024.2-network.xml.gz"
df_network = networkInfoToDf(pathToNetwork)
df_network = df_network[(df_network['type']== "highway.secondary") | (df_network['type']== "highway.tertiary") | (df_network['type']== "highway.residential")]
relevant_links = set(df_network['link_id'])

In [ ]:
pathToEvents100pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToEvents100pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/Link_enter_100pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "vehicle", "link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "entered link"):
                link_id = elem.attrib['link']
                if("pt_" in elem.attrib['link']):
                    continue
                if relevant_links and link_id not in relevant_links:
                    continue
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)

        root.clear()

In [16]:
pathToPlans100pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans100pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/Link_leave_100pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "vehicle", "link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "left link"):
                link_id = elem.attrib['link']
                if("pt_" in elem.attrib['link']):
                    continue
                if relevant_links and link_id not in relevant_links:
                    continue
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)

        root.clear()

In [ ]:
arrivals = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/arrivals_100pct.csv')
departures = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/departures_100pct.csv')

arrivals = arrivals[(arrivals['link'].str.contains("pt_")) == False]
departures = departures[(departures['link'].str.contains("pt_")) == False]

arr = arrivals.reset_index().rename(columns={'index': 'arr_idx'})
dep = departures.reset_index().rename(columns={'index': 'dep_idx'})

dep['trip_order'] = dep.groupby('person').cumcount()
arr['trip_order'] = arr.groupby('person').cumcount()
df_dep_arr = pd.merge(
    dep,
    arr,
    on=['person', 'trip_order'],
    suffixes=('_dep', '_arr')
)

df_dep_arr['car_trip_id'] = df_dep_arr.groupby('person').cumcount().add(1).astype(str)
df_dep_arr['car_trip_id'] = df_dep_arr['person'] + "_" + df_dep_arr['car_trip_id']
df_dep_arr['vehicle_id'] = df_dep_arr['person'] + "_car"
df_dep_arr = df_dep_arr[['person', 'vehicle_id', 'car_trip_id',
                        'time_dep', 'link_dep', 'time_arr', 'link_arr']].copy()

df_dep_arr = df_dep_arr.rename(columns={
    'time_dep': 'dep_time',
    'link_dep': 'dep_link',
    'time_arr': 'arr_time',
    'link_arr': 'arr_link'
})

df_dep_arr['travel_time_car'] = df_dep_arr['arr_time'] - df_dep_arr['dep_time']

In [5]:
df_Routes = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/lausitz-v2024.2-100.0-pct-plans.csv", sep = ",", names = ['person', 'vehicle_id', 'trip_id', 'start_link', 'end_link', 'route_link'])

In [7]:
def DepAndArrivalsToRoute(df_TripFromPlan, df_travTime):
    duckdb_conn = duckdb.connect() 
    duckdb_conn.register("df_TripFromPlan", df_TripFromPlan) 
    duckdb_conn.register("df_travTime", df_travTime) 

    #     m.trav_time_in_exp_plan_seconds,
    df_result= duckdb_conn.query("""SELECT 
        m.person,
        m.vehicle_id,
        m.trip_id,
        m.start_link,
        m.end_link,
        m.route_link,
        s1.dep_time,
        s1.arr_time,
        s1.travel_time_car                                             
    FROM df_TripFromPlan m
    LEFT JOIN df_travTime s1 
        ON m.trip_id = s1.car_trip_id
        AND m.vehicle_id = s1.Vehicle_id
        AND m.start_link = s1.dep_link
        AND m.end_link = s1.arr_link ;
                        """).to_df()
    duckdb_conn.close()
    return df_result


In [7]:
df_Route_trav_time = DepAndArrivalsToRoute(df_Routes, df_dep_arr)

In [9]:
df_Route_trav_time.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/df_routes_dep_and_arr.csv", index = False)

In [5]:
df_Route_trav_time = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/df_routes_dep_and_arr.csv")

In [6]:
df_Route_trav_time = pd.merge(df_Route_trav_time, df_network[['link_id', 'length', 'freespeed', 'type']], left_on="route_link", right_on="link_id", how = "left")

In [7]:
df_Route_trav_time.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/df_routes_dep_and_arr.csv", index = False)

## 50 pct

In [4]:
pathToPlans50pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans50pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/departures_50pct_case_1.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "person", "link", "legMode"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "departure" and elem.attrib['legMode'] == "car"):
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)
            
        root.clear()

In [5]:
pathToPlans50pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans50pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/arrivals_50pct_case_1.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "person", "link", "legMode"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "arrival" and elem.attrib['legMode'] == "car"):
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)
            
        root.clear()

In [8]:
pathToEvents50pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToEvents50pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/Link_enter_50pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "vehicle", "link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "entered link"):
                link_id = elem.attrib['link']
                if("pt_" in elem.attrib['link']):
                    continue
                if relevant_links and link_id not in relevant_links:
                    continue
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)

        root.clear()

In [9]:
pathToPlans50pct = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml"
tree = ET.iterparse(pathToPlans50pct, events=["start", "end"])
_, root = next(tree)

with open('/home/lola/Nextcloud/Masterarbeit/03_Outputs/Link_leave_50pct.csv', 'w', newline='') as csvfile:
    fieldnames = ["time","type", "vehicle", "link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for event, elem in tree:

        if event == "end" and elem.tag == "event":
            if(elem.attrib['type'] == "left link"):
                link_id = elem.attrib['link']
                if("pt_" in elem.attrib['link']):
                    continue
                if relevant_links and link_id not in relevant_links:
                    continue
                myDict = {key: elem.attrib[key] for key in fieldnames}
                writer.writerow(myDict)

        root.clear()

In [4]:
arrivals = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/arrivals_50pct_case_1.csv')
departures = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/departures_50pct_case_1.csv')

arrivals = arrivals[(arrivals['link'].str.contains("pt_")) == False]
departures = departures[(departures['link'].str.contains("pt_")) == False]

arr = arrivals.reset_index().rename(columns={'index': 'arr_idx'})
dep = departures.reset_index().rename(columns={'index': 'dep_idx'})

dep['trip_order'] = dep.groupby('person').cumcount()
arr['trip_order'] = arr.groupby('person').cumcount()
df_dep_arr = pd.merge(
    dep,
    arr,
    on=['person', 'trip_order'],
    suffixes=('_dep', '_arr')
)

df_dep_arr['car_trip_id'] = df_dep_arr.groupby('person').cumcount().add(1).astype(str)
df_dep_arr['car_trip_id'] = df_dep_arr['person'] + "_" + df_dep_arr['car_trip_id']
df_dep_arr['vehicle_id'] = df_dep_arr['person'] + "_car"
df_dep_arr = df_dep_arr[['person', 'vehicle_id', 'car_trip_id',
                        'time_dep', 'link_dep', 'time_arr', 'link_arr']].copy()

df_dep_arr = df_dep_arr.rename(columns={
    'time_dep': 'dep_time',
    'link_dep': 'dep_link',
    'time_arr': 'arr_time',
    'link_arr': 'arr_link'
})

df_dep_arr['travel_time_car'] = df_dep_arr['arr_time'] - df_dep_arr['dep_time']

In [5]:
df_Routes = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/lausitz-v2024.2-50.0-pct-plans-case-1.csv", sep = ",", names = ['person', 'vehicle_id', 'trip_id', 'start_link', 'end_link', 'route_link'])

In [8]:
df_Route_trav_time_50pct = DepAndArrivalsToRoute(df_Routes, df_dep_arr)

In [9]:
df_Route_trav_time_50pct = pd.merge(df_Route_trav_time_50pct, df_network[['link_id', 'length', 'freespeed', 'type']], left_on="route_link", right_on="link_id", how = "left")

In [10]:
df_Route_trav_time_50pct.to_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/df_routes_dep_and_arr_50pct_case1.csv", index = False)